In [84]:
# Enable Intellisense
%config IPCompleter.greedy=True
%load_ext autotime

import keras
from keras.wrappers.scikit_learn import KerasClassifier
from keras.datasets import mnist
from sklearn.model_selection import GridSearchCV

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
source_images = train_images
source_test_images = test_images

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 274 ms


In [85]:
# This is the training set:
train_images.shape

(60000, 28, 28)

time: 4.34 ms


In [86]:
# This is the label set:
train_labels.shape

(60000,)

time: 12.8 ms


# Network architecture

In [87]:
from keras import models, layers

# Create function returning a compiled network
def create_network(optimizer='rmsprop', activation_function='relu'):
    
    # Start neural network
    network = models.Sequential()

    # Add fully connected layer
    network.add(layers.Dense(512, activation=activation_function, input_shape=(28 * 28,)))

    # Add output layer
    network.add(layers.Dense(10, activation='softmax'))

    # Compile neural network
    network.compile(optimizer=optimizer,
               loss='categorical_crossentropy',
               metrics=['accuracy'])
    
    # Return compiled network
    return network

# Wrap function in KerasClassifier for GridSearch
neural_network = KerasClassifier(build_fn=create_network, verbose=0)

time: 2.73 ms


## Feature scaling

Data is a 3-dimension array of (6000, 28, 28). Data samples are values ranging between [0, 255]. We need to transform those values to range [0, 1]

In [88]:
# Convert 2-dimensional image into 1-dimensional array
train_images = train_images.reshape((60000, 28 * 28))

# Normalize
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

time: 89.6 ms


In [89]:
# Categorically encode the labels

from keras.utils import to_categorical

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

time: 2.72 ms


## Prepare GridSearch

In [90]:
optimizers = ['rmsprop', 'adam', 'adagrad', 'adamax']
activation_functions = ['relu', 'elu', 'selu']
epochs = [5]
batches = [128]

# Create hyperparameter options
hyperparameters = dict(optimizer=optimizers, activation_function=activation_functions, epochs=epochs, batch_size=batches)

grid = GridSearchCV(estimator=neural_network, cv=3, param_grid=hyperparameters)

time: 2.21 ms


## Fit the network

In [ ]:
grid_result = grid.fit(train_images, train_labels)

## Best scores

In [ ]:
grid_result.best_params_

## Prediction

Pick a random number from the test set. We will ask the network to predict it.

In [ ]:
index_test = random.randint(0, 9999)
test_number = source_test_images[index_test]
test_image = test_images[index_test]
plt.imshow(test_number, cmap=plt.cm.binary)
plt.show()

In [ ]:
import numpy as np

network.predict(np.reshape(test_image, (1, 28 * 28)))[0]